# Interactive Doctorates Dashboard  

## PhDs Awarded in the US

By: Dapo Adegbile

### In this blog post I will be examining how many and the types of Doctorates given out in the United States in 2017. The data is originally taken from: https://ncses.nsf.gov/pubs/nsf19301/data. I cleaned the dataset and perfomed some analysis using `pandas` , and I then made a dashboard to better visualize key insights from the data in plotly and Streamlit. 

### I chose to use Plotly as I am already familar with it's visualizations and I choose to use streamlit to deploy the dashboard because of the ease of use and the sleak display of the dashboard.


## Data Cleaning

### My data cleaning process can be found below:

In [24]:
#collapse
import pandas as pd
import numpy as np

In [25]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [26]:
#collapse
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]


In [27]:
#collapse
df = pd.read_excel("/Users/dapoadegbile/Documents/data_tables/sed17-sr-tab007.xlsx", header = 4)




In [28]:
#collapse
df1 = df.set_index(df.iloc[:,0])


In [29]:
#collapse
df1 = df1.loc[state_names]


In [30]:
#collapse
df1 = df1.reset_index(drop = True)


In [31]:
#collapse

df_final = df1[['Unnamed: 0', 'Unnamed: 1', 'Life sciences', 
     'Physical sciences and earth sciences',
     'Mathematics and computer sciences', 'Psychology and social sciences ', 'Unnamed: 20' ]]


In [32]:
#collapse

df_final.columns = ['State', "All Fields" , 'Life Sciences' ,
                   'Physical Sciences and Earth Sciences' , 'Mathematics and Computer Sciences' ,
                    'Psychology and Social Sciences' , 'Engineering']


In [33]:
#collapse
df_exports = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

df_codes = df_final.merge(df_exports, left_on = 'State', right_on = 'state', how = 'left')


In [34]:
#collapse
df_codes = df_codes.iloc[:,0:8]

df_codes.to_csv('/Users/dapoadegbile/Documents/Biostat 823/Homework/DoctoratesbyStateandDegree.csv', index_label = False)

### A snippet of my final dataset can be found below to see what it looks like after it's cleaned:

In [35]:
#collapse
df_codes.head(15)

,State,All Fields,Life Sciences,Physical Sciences and Earth Sciences,Mathematics and Computer Sciences,Psychology and Social Sciences,Engineering,code
0,Alaska,52.0,10.0,20.0,0.0,17.0,3.0,AK
1,Alabama,707.0,188.0,59.0,48.0,89.0,131.0,AL
2,Arkansas,202.0,75.0,17.0,10.0,19.0,33.0,AR
3,Arizona,801.0,133.0,110.0,51.0,156.0,141.0,AZ
4,California,6105.0,1300.0,841.0,495.0,1132.0,1151.0,CA
5,Colorado,1005.0,212.0,155.0,67.0,128.0,206.0,CO
6,Connecticut,748.0,196.0,104.0,60.0,129.0,97.0,CT
7,District of Columbia,623.0,55.0,37.0,24.0,207.0,78.0,NaN
8,Delaware,238.0,59.0,24.0,19.0,45.0,69.0,DE
9,Florida,2336.0,490.0,259.0,145.0,409.0,419.0,FL


### Below I've added the code that I used to create my dashboard which can be found below: 

### https://share.streamlit.io/dapoade/doctorate-dashboards/main/DoctoratesDashboard.py

### Using a choropleth map, we can cleanly see how the popularity of each field varies from state to state. The total number of schools is represented by the color gradient, and you can filter the fields to see which fields are more popular within each state. Additionally I added a bar chart that shows each fields popularity in each state as another means of comparison. 

### From the dashboard we can see that California gave out the most doctorate degrees in 2017, followed by Texas and New York. Additionally LIfe Sciences is the hottest field, while Engineering and Psychology and Social Sciences are a close second and third. 



In [36]:
#collapse_output
import plotly.graph_objects as go

for col in df_codes.columns:
    df_codes[col] = df_codes[col].astype(str)

# df_codes['text'] = df_codes['All Fields'] + '<br>' + \
#     'Life Sciences ' + df_codes['Life Sciences'] + ' Physical Sciences and Earth Sciences ' + df_codes['Physical Sciences and Earth Sciences'] + '<br>' + \
#     'Mathematics and Computer Sciences ' + df_codes['Mathematics and Computer Sciences'] + ' Psychology and Social Sciences ' + df_codes['Psychology and Social Sciences'] + '<br>' + \
#     'Engineering ' + df_codes['Engineering']




fig = go.Figure(data=go.Choropleth(
    locations= df_codes['code'], # Spatial coordinates
    z = df_codes['All Fields'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Greens',
#     hover_name = "State", hover_data = "All Fields",
    text=df_codes['State'], # hover text
    colorbar_title = "Number of Doctorates",
))

fig.update_layout(
    title_text = 'Number of Doctorates by State',
    geo_scope='usa', # limit map scope to USA
)

fig.show()

In [37]:
#collapse_output
fig = go.Figure(go.Bar(
            y=(df_codes['All Fields']).astype(float),
            x= df_codes['State'],
            marker_color = 'indianred'))
fig.update_layout(xaxis_tickangle=-90)

fig.show()